In [43]:
from itertools import cycle
import pandas as pd
import os
import json

In [6]:
files = [file for file in os.listdir('conjugations') if file.endswith('.csv')]

In [53]:
_columns = {
    'pess': 'person',
    'conj': 'conjugation'
}

_person = {
    'eu': '1s',
    'tu': '2s',
    'ele/ela': '3s',
    'nós': '1p',
    'vós': '2p',
    'eles/elas': '3p'
}

_time = {
    'futuro': 'fdpres',
    'condicional': 'fdpret',
    'pret. mais-que-perfeito': 'pmpf',
    'presente': 'p',
    'pretérito perfeito': 'pp',
    'pretérito imperfeito': 'pi'
}

_mode = {
    'indicativo': 'ind'
}

_person_filter = ['1s', '3s', '1p', '3p']

conjugations = (pd.concat([pd.read_csv(f'conjugations/{file}') for file in files])
                  .rename(columns=_columns)
                  .assign(person=lambda df: df.person.map(_person),
                          time=lambda df: df.time.map(_time),
                          mode=lambda df: df['mode'].map(_mode))
                  .query('person == @_person_filter'))

conjugations.head(20)

,conjugation,infinitive,mode,person,time
0,sou,ser,ind,1s,p
1,fui,ser,ind,1s,pp
2,era,ser,ind,1s,pi
6,é,ser,ind,3s,p
7,foi,ser,ind,3s,pp
8,era,ser,ind,3s,pi
9,somos,ser,ind,1p,p
10,fomos,ser,ind,1p,pp
11,éramos,ser,ind,1p,pi
15,são,ser,ind,3p,p


In [54]:
conjugations.time.unique()

array(['p', 'pp', 'pi', 'pmpf', 'fdpres', 'fdpret'], dtype=object)

In [55]:
conjugations['mode'].unique()

array(['ind'], dtype=object)

In [56]:
conj = {}

for verb, verb_df in conjugations.groupby('infinitive'):
    conj[verb] = {}
    for mode, mode_df in verb_df.groupby('mode'):
        conj[verb][mode] = {}
        for time, time_df in mode_df.groupby('time'):
            conj[verb][mode][time] = {}
            for pers, pers_df in time_df.groupby('person'):
                conj[verb][mode][time][pers] = pers_df.conjugation.values[0]

In [57]:
conj

{'abandonar': {'ind': {'fdpres': {'1p': 'abandonaremos',
    '3p': 'abandonarão',
    '3s': 'abandonará'},
   'fdpret': {'1p': 'abandonaríamos',
    '3p': 'abandonariam',
    '3s': 'abandonaria'},
   'p': {'1p': 'abandonamos',
    '1s': 'abandono',
    '3p': 'abandonam',
    '3s': 'abandona'},
   'pi': {'1p': 'abandonávamos',
    '1s': 'abandonava',
    '3p': 'abandonavam',
    '3s': 'abandonava'},
   'pmpf': {'1p': 'abandonáramos', '3p': 'abandonaram', '3s': 'abandonara'},
   'pp': {'1p': 'abandonámos / abandonamos',
    '1s': 'abandonei',
    '3p': 'abandonaram',
    '3s': 'abandonou'}}},
 'abrir': {'ind': {'fdpres': {'1p': 'abriremos',
    '1s': 'abrirei',
    '3p': 'abrirão',
    '3s': 'abrirá'},
   'fdpret': {'1p': 'abriríamos',
    '1s': 'abriria',
    '3p': 'abririam',
    '3s': 'abriria'},
   'p': {'1p': 'abrimos', '1s': 'abro', '3p': 'abrem', '3s': 'abre'},
   'pi': {'1p': 'abríamos', '1s': 'abria', '3p': 'abriam', '3s': 'abria'},
   'pmpf': {'1p': 'abríramos',
    '1s': 'abri

In [58]:
with open('conjugations.json', 'w') as file:
    json.dump(conj, file, indent=2)

In [51]:
with open('conjugations.json', 'r') as file:
    rconj = json.loads(file.read())

In [52]:
rconj

{'abandonar': {'ind': {'fdpres': {'1p': 'abandonaremos',
    '3p': 'abandonarão',
    '3s': 'abandonará'},
   'fdpret': {'1p': 'abandonaríamos',
    '3p': 'abandonariam',
    '3s': 'abandonaria'},
   'pi': {'1p': 'abandonávamos',
    '1s': 'abandonava',
    '3p': 'abandonavam',
    '3s': 'abandonava'},
   'pmpf': {'1p': 'abandonáramos', '3p': 'abandonaram', '3s': 'abandonara'},
   'pp': {'1p': 'abandonamos',
    '1s': 'abandono',
    '3p': 'abandonam',
    '3s': 'abandona'}}},
 'abrir': {'ind': {'fdpres': {'1p': 'abriremos',
    '1s': 'abrirei',
    '3p': 'abrirão',
    '3s': 'abrirá'},
   'fdpret': {'1p': 'abriríamos',
    '1s': 'abriria',
    '3p': 'abririam',
    '3s': 'abriria'},
   'pi': {'1p': 'abríamos', '1s': 'abria', '3p': 'abriam', '3s': 'abria'},
   'pmpf': {'1p': 'abríramos',
    '1s': 'abrira',
    '3p': 'abriram',
    '3s': 'abrira'},
   'pp': {'1p': 'abrimos', '1s': 'abro', '3p': 'abrem', '3s': 'abre'}}},
 'acabar': {'ind': {'fdpres': {'1p': 'acabaremos',
    '1s': 'acab

In [59]:
verbs = pd.read_csv('top_1000_verbs_pt.csv').verbs.values
verbs

array(['ser', 'ter', 'estar', 'poder', 'fazer', 'ir', 'haver', 'dizer',
       'dar', 'ver', 'saber', 'querer', 'ficar', 'dever', 'passar', 'vir',
       'chegar', 'falar', 'deixar', 'encontrar', 'levar', 'começar',
       'partir', 'pensar', 'parecer', 'apresentar', 'olhar', 'tornar',
       'sair', 'voltar', 'conseguir', 'achar', 'existir', 'sentir',
       'entrar', 'chamar', 'conhecer', 'considerar', 'pôr', 'continuar',
       'viver', 'ouvir', 'tomar', 'acabar', 'receber', 'perder', 'andar',
       'trabalhar', 'criar', 'pedir', 'seguir', 'contar', 'acontecer',
       'afirmar', 'tratar', 'esperar', 'gostar', 'usar', 'manter',
       'realizar', 'abrir', 'escrever', 'permitir', 'ocorrer', 'mostrar',
       'lembrar', 'trazer', 'procurar', 'morrer', 'tentar', 'formar',
       'aparecer', 'incluir', 'cair', 'correr', 'ganhar', 'surgir',
       'nascer', 'pagar', 'representar', 'entender', 'produzir', 'ler',
       'precisar', 'perguntar', 'constituir', 'colocar', 'possuir',
       '

In [62]:
pd.DataFrame([verb for verb in verbs if verb in conj]).to_csv('verbs.csv', header=False, index=False)

In [104]:
conjugations = pd.read_csv('top_100_pt.csv')
conjugations.head()

,conj,infinitive,mode,pess,time
0,sou,ser,indicativo,eu,presente
1,fui,ser,indicativo,eu,pretérito perfeito
2,era,ser,indicativo,eu,pretérito imperfeito
3,és,ser,indicativo,tu,presente
4,foste,ser,indicativo,tu,pretérito perfeito


In [118]:
conj['ser']['indicativo']['presente']

{'ele/ela': 'é',
 'eles/elas': 'são',
 'eu': 'sou',
 'nós': 'somos',
 'tu': 'és',
 'vós': 'sois'}

In [129]:
ask_verb('ser', 'indicativo', 'presente')

conjugate ser
eu: sou
ele/ela: 
nós: somos
eles/elas: são


False

In [115]:
conj['ser']['indicativo']['presente']['eu']

'sou'

In [97]:
class Reviewer:
    def __init__(self):

def get_seq(i):
    nums = ''.join([str(i) for i in range(10)])
    out = [nums[i]]
    for d in range(2, 5):
        out.append(nums[(i + d) % 10])
        i += d
    return ''.join(out)


def put_in_progress(decks, session, word):
    decks['progress'][get_seq(session)].append(word)


def remove_from_progress(decks, session, word):
    decks['progress'][get_seq((session + 1) % 10)].remove(word)

    
def remove_from_current(decks, word):
    decks['current'].remove(word)
    
    
def put_in_retired(decks, word):
    decks['retired'].append(word)
    

def create_decks():
    return {
        'current':list(verbs[:10]),
        'progress': { get_seq(i): [] for i in range(10)},
        'retired': []
    }


def ask_verb(verb, mode, time):
    persons = ('eu', 'ele/ela', 'nós', 'eles/elas')
    print(f"conjugate {verb}")
    answers = [input(f"{person}: ") == conj[verb][mode][time][person] for person in persons]
    return all(answers)

In [98]:
decks

{'current': ['ser',
  'ter',
  'estar',
  'poder',
  'fazer',
  'ir',
  'haver',
  'dizer',
  'dar',
  'ver'],
 'progress': {'0259': [],
  '1360': [],
  '2471': [],
  '3582': [],
  '4693': [],
  '5704': [],
  '6815': [],
  '7926': [],
  '8037': [],
  '9148': []},
 'retired': []}

In [99]:
put_in_progress(decks, 0, 'ser')
remove_from_current(decks, 'ser')

In [100]:
decks

{'current': ['ter',
  'estar',
  'poder',
  'fazer',
  'ir',
  'haver',
  'dizer',
  'dar',
  'ver'],
 'progress': {'0259': ['ser'],
  '1360': [],
  '2471': [],
  '3582': [],
  '4693': [],
  '5704': [],
  '6815': [],
  '7926': [],
  '8037': [],
  '9148': []},
 'retired': []}

In [69]:
''.join([str(i) for i in range(10)])

'0123456789'

In [131]:
from termcolor import colored

In [133]:
print(colored('holi', 'red'), colored('chau', 'green'))

holi chau


In [134]:
import random

In [147]:
random.sample([1, 2, 3], 3)

[3, 1, 2]